In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothic'

matplotlib.rc("axes", unicode_minus = False)

In [3]:
df1 = pd.read_csv("01 Oxidation.csv")
df2 = pd.read_csv("02 Photo_softbake.csv")
df3 = pd.read_csv("03 Photo_lithograpy.csv")
df4 = pd.read_csv("04 Etching.csv")
df5 = pd.read_csv("05 Ion_Implantation.csv")
df6 = pd.read_csv("06 Inspect.csv")

In [4]:
df = pd.merge(df1, df2, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df3, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df4, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df5, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])
df = pd.merge(df, df6, on=['No_Die','Lot_Num', 'Wafer_Num','Datetime'])

In [5]:
df.dropna(subset="Thin F2", inplace=True) # 다른 변수들과의 연관성을 찾지못함, 한 행에 여러 열들 결측값 가짐
# pd.set_option('display.max_row', 200)
# pd.set_option('display.max_columns', 200)

In [6]:
df['Ox_Chamber'] = df['Ox_Chamber'].astype('str')
df['photo_soft_Chamber'] = df['photo_soft_Chamber'].astype('str')
df['lithography_Chamber'] = df['lithography_Chamber'].astype('str')
df['Etching_Chamber'] = df['Etching_Chamber'].astype('str')
df['Chamber_Num'] = df['Chamber_Num'].astype('str')
df['path'] = df['Ox_Chamber']+df['photo_soft_Chamber']+df['lithography_Chamber']+df['Etching_Chamber']+df['Chamber_Num']

In [7]:
df.drop(df[df['Oxid_time']<0].index, inplace=True) # 산화시간이 음수
df.drop(df[df['Target']==0].index, inplace=True) # target 값이 0
df.drop(columns = ["Wafer_map","Error_message"],inplace=True) # 웨이퍼맵, 에러메시지

In [8]:
# 시간 데이터 datetime 유형으로 변환
df['Datetime'] = pd.to_datetime(df['Datetime'], format='%d-%m-%Y')

# 195이상 = 불량(1), 195미만 = 양품(0)
df.loc[df['Target'] >= 195, '불량_195이상'] =1
df.loc[df['Target'] < 195, '불량_195이상'] =0

In [9]:
df.loc[df['Thin F4']<0, 'Thin F4']=df['Thin F4'].median()
df.loc[df['Flux90s']<0, 'Flux90s']=df['Flux90s'].median()
df.loc[df['Flux160s']<0, 'Flux160s']=df['Flux160s'].median()
df.loc[df['Flux160s']<5, 'Flux160s']=df['Flux160s'].median()

In [10]:
df['Thin F1'].fillna(df['Thin F1'].median(), inplace=True)
df['Thin F3'].fillna(df['Thin F3'].median(), inplace=True)
df['Flux60s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux90s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux480s'].fillna(df['Flux90s'].median(), inplace=True)
df['Flux840s'].fillna(df['Flux90s'].median(), inplace=True)

### 불필요한 열 제거

In [11]:
# 불필요한 열 제거 
# df.drop(columns=["No_Die", "Lot_Num", "Wafer_Num"], inplace=True)
df.drop(columns=['Vapor','process','Wavelength'])
df.loc[df['Flux840s'] == 8.137500e+16, 'Flux840s'] = df['Flux840s'].mode()[0]
df.loc[df['Flux480s'] == 8.137500e+16, 'Flux480s'] = df['Flux480s'].mode()[0]

In [12]:
df_new = df[['Temp_OXid','ppm','Pressure','type','Oxid_time',
        'N2_HMDS', 'pressure_HMDS', 'temp_HMDS', 'temp_HMDS_bake',
       'time_HMDS_bake', 'spin1', 'spin2', 'spin3', 'photoresist_bake',
       'temp_softbake', 'time_softbake', 'UV_type', 'Energy_Exposure',       
        'Temp_Etching', 'Source_Power',
       'input_Energy', 'Temp_implantation', 'Furance_Temp', 'RTA_Temp',
       '불량_195이상', 'path']]

In [13]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1693 entries, 0 to 1703
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Temp_OXid          1693 non-null   float64
 1   ppm                1693 non-null   float64
 2   Pressure           1693 non-null   float64
 3   type               1693 non-null   object 
 4   Oxid_time          1693 non-null   int64  
 5   N2_HMDS            1693 non-null   float64
 6   pressure_HMDS      1693 non-null   float64
 7   temp_HMDS          1693 non-null   float64
 8   temp_HMDS_bake     1693 non-null   float64
 9   time_HMDS_bake     1693 non-null   float64
 10  spin1              1693 non-null   float64
 11  spin2              1693 non-null   float64
 12  spin3              1693 non-null   float64
 13  photoresist_bake   1693 non-null   float64
 14  temp_softbake      1693 non-null   float64
 15  time_softbake      1693 non-null   float64
 16  UV_type            1693 non-n

In [16]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

df_raw_x = df_new.drop("불량_195이상", axis = 1)
df_raw_y = df_new["불량_195이상"] 

# MinMaxScaler 객체 생성
numeric_columns = df_raw_x.select_dtypes(include=['number']).columns
scaler = MinMaxScaler()

df_raw_x[numeric_columns] = scaler.fit_transform(df_raw_x[numeric_columns])

df_raw_x = pd.get_dummies(df_raw_x, columns=df_raw_x.select_dtypes(include=['object']).columns)
df_raw_x = df_raw_x.astype({column: int for column in df_raw_x.select_dtypes(include=['bool']).columns})

df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
    df_raw_x, df_raw_y, test_size = 0.3, random_state = 1234, stratify=df_raw_y) 

In [17]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 데이터 구성:Series, DataFrame
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib

# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV

# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [20]:
import optuna
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [26]:
lgb = LGBMClassifier(n_estimators=100,is_unbalance = True,boost_from_average=False)
lgb.fit(df_train_x, df_train_y)
y_pred = lgb.predict(df_test_x)
print(confusion_matrix(df_test_y, y_pred))
print(classification_report(df_test_y, y_pred))

[LightGBM] [Info] Number of positive: 85, number of negative: 1100
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001225 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4172
[LightGBM] [Info] Number of data points in the train set: 1185, number of used features: 53
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[[467   5]
 [ 11  25]]
              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98       472
         1.0       0.83      0.69      0.76        36

    accuracy                           0.97       508
   macro avg       0.91      0.84      0.87       508
weighted avg       0.97      0.97      0.97       508



In [ ]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Define the parameter grid
param_dist = {
    'n_estimators': sp_randint(50, 200),  # Number of trees
    'num_leaves': sp_randint(10, 100),  # Maximum number of leaves in one tree
    'learning_rate': [0.001, 0.01, 0.1],  # Learning rate
    'subsample': [0.5, 0.7, 0.9],  # Subsample ratio of the training instance
    'colsample_bytree': [0.5, 0.7, 0.9],  # Subsample ratio of columns when constructing each tree
    'reg_alpha': [0, 0.1, 0.5, 1],  # L1 regularization term on weights
    'reg_lambda': [0, 0.1, 0.5, 1]  # L2 regularization term on weights
}

# Create a LGBMClassifier object
lgb = LGBMClassifier(is_unbalance=True, boost_from_average=False)

# Randomized search with 10-fold cross-validation
random_search = RandomizedSearchCV(
    estimator=lgb,
    param_distributions=param_dist,
    n_iter=100,  # Number of parameter settings that are sampled
    cv=10,  # 10-fold cross-validation
    scoring='accuracy',  # Evaluation metric
    n_jobs=-1,  # Use all available cores
    verbose=1,
    random_state=42
)

# Fit the random search model
random_search.fit(df_train_x, df_train_y)

# Print the best parameters found
print("Best parameters found: ", random_search.best_params_)

# Use the best estimator to make predictions
y_pred = random_search.best_estimator_.predict(df_test_x)

# Print confusion matrix and classification report
print(confusion_matrix(df_test_y, y_pred))
print(classification_report(df_test_y, y_pred))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[LightGBM] [Info] Number of positive: 77, number of negative: 990
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.674985 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4129
[LightGBM] [Info] Number of data points in the train set: 1067, number of used features: 42
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai